In [60]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [61]:
# Скачиваем исходные файлы
!gdown 1ERwQ5odiK1Zvi1LtjpkzCMUswYsAX8_K  # train.csv
!gdown 1fGw_-RFwvn_LEdt91Jq-7A-wzG6mmH8r  # test.csv
!gdown 199Mt4OYZNaelT83U-HGDsEYs2YcUGQ6y  # submission.csv

Downloading...
From: https://drive.google.com/uc?id=1ERwQ5odiK1Zvi1LtjpkzCMUswYsAX8_K
To: /content/train.csv
100% 664k/664k [00:00<00:00, 21.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fGw_-RFwvn_LEdt91Jq-7A-wzG6mmH8r
To: /content/test.csv
100% 218k/218k [00:00<00:00, 16.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=199Mt4OYZNaelT83U-HGDsEYs2YcUGQ6y
To: /content/submission.csv
100% 14.7k/14.7k [00:00<00:00, 61.3MB/s]


In [62]:
# Загружаем тренировочный датасет
data = pd.read_csv('./train.csv')

# Приводим колонки к числовому значению
data['MonthlySpending'] = pd.to_numeric(data['MonthlySpending'], errors='coerce')
data['TotalSpent'] = pd.to_numeric(data['TotalSpent'], errors='coerce')

# Заменяем медианой пропущенные значения в 'TotalSpent'
median_value = data['TotalSpent'].median()
data['TotalSpent'].fillna(median_value, inplace=True)

In [63]:
# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'

**1. ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ**

ROC-AUC на валидационной выборке: 0.82686

In [64]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [65]:
# 1. Нормализация числовых признаков
data1 = data # делаем копию для логистической регрессии, чтобы не изменяла датафрейм для использования в Catboost
scaler = StandardScaler()
data1[num_cols] = scaler.fit_transform(data1[num_cols])

# 2. One-Hot Encoding для категориальных признаков
data1 = pd.get_dummies(data1, columns=cat_cols, drop_first=True)

In [66]:
from sklearn.metrics import roc_auc_score

# Разделение данных
X = data1.drop(target_col, axis=1)
y = data1[target_col]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression с автоматическим подбором параметра C
clf = LogisticRegressionCV(Cs=[100, 10, 1, 0.1, 0.01, 0.001], scoring='roc_auc', refit=True, cv=5, max_iter=1000)
clf.fit(X_train, y_train)

# Проверка на тестовой выборке
y_pred_proba = clf.predict_proba(X_valid)[:, 1]
roc_auc = roc_auc_score(y_valid, y_pred_proba)

print(f"Лучшее значение C: {clf.C_[0]}")
print(f"ROC-AUC на валидационной выборке: {roc_auc}")

Лучшее значение C: 100.0
ROC-AUC на валидационной выборке: 0.8268578975074112


**2. CATBOOST**

ROC-AUC на валидационной выборке: 0.8244

In [67]:
!pip install catboost # установка catboost, в Colab не установлена по умолчанию

In [68]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [69]:
data2 = data  # делаем копию датафрейма для catboost

In [70]:
# Разделение данных
X_1 = data2[feature_cols]
y_1 = data2[target_col]
X_train_cb, X_valid_cb, y_train_cb, y_valid_cb = train_test_split(X_1, y_1, test_size=0.2, random_state=42)

# Создание и обучение модели CatBoost
catboost_model = CatBoostClassifier(verbose=500)  # verbose=500, чтобы выводить прогресс каждые 500 итераций
catboost_model.fit(X_train_cb, y_train_cb, cat_features=cat_cols)

# Предсказание на валидационной выборке
y_pred_proba_cb = catboost_model.predict_proba(X_valid_cb)[:, 1]

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_valid_cb, y_pred_proba_cb)
print(f"ROC-AUC на валидационной выборке: {roc_auc}")

Learning rate set to 0.019062
0:	learn: 0.6798215	total: 18.7ms	remaining: 18.6s
500:	learn: 0.3555245	total: 5.31s	remaining: 5.29s
999:	learn: 0.3132948	total: 16.9s	remaining: 0us
ROC-AUC на валидационной выборке: 0.8244359582936988


**ИСПОЛЬЗОВАНИЕ МОДЕЛИ**

In [74]:
# Пробуем модель с наилучшим результатом, это логистическая регрессия
best_model = clf

X_test = pd.read_csv('./test.csv')
submission = pd.read_csv('./submission.csv')

In [75]:
# Удаление строк с пропущенными значениями в 'TotalSpent'
X_test['MonthlySpending'] = pd.to_numeric(X_test['MonthlySpending'], errors='coerce')
X_test['TotalSpent'] = pd.to_numeric(X_test['TotalSpent'], errors='coerce')
median_value = X_test['TotalSpent'].median()
X_test['TotalSpent'].fillna(median_value, inplace=True)

# Применение one-hot-encoding к категориальным признакам
X_test = pd.get_dummies(X_test, columns=cat_cols)

# Проверка на наличие всех необходимых колонок
missing_cols = set(X_train.columns) - set(X_test.columns)
for c in missing_cols:
    X_test[c] = 0
X_test = X_test[X_train.columns]

# Масштабирование численных признаков
X_test[num_cols] = scaler.transform(X_test[num_cols])

# Предсказание вероятностей с помощью best_model
submission['Churn'] = best_model.predict_proba(X_test)[:, 1]

# Создание файла my_submission.csv
submission.to_csv('./my_submission.csv', index=False)